In [1]:
import os
os.chdir('..')

import bookworm.data
import  altair as alt

# Pull Data

In [2]:
df = bookworm.data.get_checkouts_by_usage_class()

Client created, querying data...
Raw data queried, processing data...
Done


In [3]:
df.head()

,month_date,usage_class,monthly_checkouts
58,2017-01-01,Digital,185376
36,2017-01-01,Physical,331830
31,2017-02-01,Digital,168370
66,2017-02-01,Physical,299992
56,2017-03-01,Digital,186795


# Digital vs Physical Trends

Between January 2017 and March 2020, Digital checkouts grew linearly each month while Physical checkouts stayed approximately constant with some month-to-month variation. After March 2020, Physical checkouts dropped to zero due to branch closures, and Digital checkouts seemed to increase beyond the expected trend. How much did Digital checkouts increase in April, May and June 2020 over what was expected due to Covid-19?

In [10]:
df.loc[:, 'usage_class'] = df['usage_class'].astype('str')

In [13]:
df = df.astype(dtype={'usage_class':'str'})

In [15]:
alt.Chart(df).mark_line().encode(
    x='month_date',
    y='monthly_checkouts',
    color='usage_class'
)

alt.Chart(...)

In [18]:
all(df.columns == ['month_date', 'usage_class', 'monthly_checkouts'])

True

In [23]:
import datetime as dt

In [31]:
df['usage_class'].unique() == ['Physical', 'Digital']

array([False, False])